# Simulate an XOR game

Referance:  
https://cirq.readthedocs.io/en/stable/examples.html#bell-inequality  
https://people.eecs.berkeley.edu/~vazirani/s07quantum/notes/lecture1.pdf

![game discription](img/game_description.png)

![strategy](img/game_strategy.png)

In [8]:
import numpy as np
import cirq

In [9]:
alice = cirq.GridQubit(0, 0)
bob = cirq.GridQubit(1, 0)
alice_referee = cirq.GridQubit(0, 1)
bob_referee = cirq.GridQubit(1, 1)

In [10]:
circuit = cirq.Circuit()

In [11]:
# Prepare shared entangled state.
circuit.append([
    cirq.H(alice),
    cirq.CNOT(alice, bob),
    cirq.X(alice)**-0.25,
])

In [12]:
# Referees flip coins.
circuit.append([
    cirq.H(alice_referee),
    cirq.H(bob_referee),
], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

In [13]:
# Players do a sqrt(X) based on their referee's coin.
circuit.append([
    cirq.CNOT(alice_referee, alice)**0.5,
    cirq.CNOT(bob_referee, bob)**0.5,
], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

In [14]:
# Then results are recorded.
circuit.append([
    cirq.measure(alice, key='a'),
    cirq.measure(bob, key='b'),
    cirq.measure(alice_referee, key='x'),
    cirq.measure(bob_referee, key='y'),
], strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

In [15]:
# Show circuit.
print(circuit)

(0, 0): ───H───@───X^-0.25───────X───────M('a')───
               │                 │
(0, 1): ───────┼─────────────H───@^0.5───M('x')───
               │
(1, 0): ───────X─────────────────X───────M('b')───
                                 │
(1, 1): ─────────────────────H───@^0.5───M('y')───


In [20]:
# Run simulations.
repetitions = 100000
print('Simulating {} repetitions...'.format(repetitions))
result = cirq.Simulator().run(program=circuit, repetitions=repetitions)

Simulating 100000 repetitions...


In [21]:
# Collect results.
a = np.array(result.measurements['a'][:, 0])
b = np.array(result.measurements['b'][:, 0])
x = np.array(result.measurements['x'][:, 0])
y = np.array(result.measurements['y'][:, 0])
outcomes = a ^ b == x & y
win_percent = len([e for e in outcomes if e]) * 100 / repetitions

In [22]:
def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)

In [23]:
# Print data.
print('a:', bitstring(a[:100]))
print('b:', bitstring(b[:100]))
print('x:', bitstring(x[:100]))
print('y:', bitstring(y[:100]))
print('(a XOR b) == (x AND y):\n  ', bitstring(outcomes[:100]))
print('Win rate: {}%'.format(win_percent))

a: _11_1_1__11___11_1___111_1111_111____11______11111111_111_11__11__11__1___1111_1111__1_11_____1_111_
b: __1_1_111____1___1_1_11__1111_111___111_1___11111___1___1_____11__11_11_1__11__1111_____1_11__1_11_1
x: _1_11_11111__11___1111___1_11_____111_1_1_111______1__11_1_11_1__1__11__1_1__11_11_1_1111111_1_11111
y: 111_11__1_1__11____1__11__1__11111_111_11____1_1_111_1111__1_1_______1_11_1__1_1____11_1_1111_____11
(a XOR b) == (x AND y):
   1111_11_1_11111_1111111_11111111111_11111111_1111__1111111_111111111111111111111111111111_1111111111
Win rate: 85.366%


In [24]:
# Theoretical win rate
np.cos(np.pi/8)**2

0.8535533905932737